# MapReduce

### Download dataset

In [1]:
!wget -O natural_disasters_BIG.csv https://www.dropbox.com/s/b74bpkbr89l8k43/natural_disasters_BIG.csv?dl=0

--2019-11-15 12:33:13--  https://www.dropbox.com/s/b74bpkbr89l8k43/natural_disasters_BIG.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.68.1, 2620:100:6024:1::a27d:4401
Connecting to www.dropbox.com (www.dropbox.com)|162.125.68.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/b74bpkbr89l8k43/natural_disasters_BIG.csv [following]
--2019-11-15 12:33:15--  https://www.dropbox.com/s/raw/b74bpkbr89l8k43/natural_disasters_BIG.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6d76ca42288fed7fd9fe75e7ba.dl.dropboxusercontent.com/cd/0/inline/AsZNHkYww8XIADG-03CfqnGhS8k2g03G0_--I8rNrjhW501tK7-tM2J62yKeFdjQjrjuKpoTYIMjFM5a8dfbCh5bT2VhOx_HcmCKDdKp12Yzcg/file# [following]
--2019-11-15 12:33:16--  https://uc6d76ca42288fed7fd9fe75e7ba.dl.dropboxusercontent.com/cd/0/inline/AsZNHkYww8XIADG-03CfqnGhS8k2g03G0_--I8rNrjhW501tK7-tM2J62yKeFdjQjrjuKpoTYIMjFM5a8dfbCh5b

### 1. How many disasters occurred in continent C?

In [2]:
%%file mapper_ex1.py
#!/usr/bin/env python

# import sys
import sys
# import string library function  
import string  

# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split into columns
    lines = line.split(",")
    # print continent and ocurrences
    print(f"{lines[5]}\t{lines[9]}")

Overwriting mapper_ex1.py


In [3]:
%%file reducer_ex1.py
#!/usr/bin/env python

import sys

# variable that will store the current iteration key
last_continent = ""
# count variable for ocurrences
count = 0

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    continent, count_line = line.split('\t')

    # check if it's the first iteration ever and update key variable
    if last_continent == "":
        last_continent = continent

    # if same key add count
    if continent == last_continent:
        count += int(count_line)

    # else print last key and refresh a new key iteration
    else:
        # print continent and ocurrences
        print(f"{last_continent}\t{count}")
        # refresh variables
        last_continent = continent
        count = int(count_line)
        
# print continent and ocurrences for last key
print(f"{last_continent}\t{count}")

Overwriting reducer_ex1.py


In [4]:
!rm -rf results_continents; bash -c 'time hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper_ex1.py,reducer_ex1.py -mapper mapper_ex1.py -reducer reducer_ex1.py -input natural_disasters_BIG.csv -output results_continents'

2019-11-15 12:37:52,114 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2019-11-15 12:37:52,418 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2019-11-15 12:37:52,419 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2019-11-15 12:37:52,581 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2019-11-15 12:37:53,764 INFO mapred.FileInputFormat: Total input files to process : 1
2019-11-15 12:37:53,843 INFO mapreduce.JobSubmitter: number of splits:7
2019-11-15 12:37:54,772 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1107095772_0001
2019-11-15 12:37:54,772 INFO mapreduce.JobSubmitter: Executing with tokens: []
2019-11-15 12:37:56,530 INFO mapred.LocalDistributedCacheManager: Localized file:/home/jovyan/work/mapper_ex1.py as file:/tmp/hadoop-jovyan/mapred/local/job_local1107095772_0001_6a138863-5bfc-4ef2-b008-16caeeb1d615/mapper_ex1.py
2019-11-15 12:37:56,642 INFO mapred.Loca

In [5]:
!cat results_continents/part-*

Africa	576686
Americas	626963
Asia	1191797
Europe	362135
Oceania	89256


### 2. In which regions there were disasters of type X?

In [6]:
%%file mapper_ex2.py
#!/usr/bin/env python

# import sys
import sys
# import string library function  
import string  

# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split into columns
    lines = line.split(",")
    # print disaster type and region
    print(f"{lines[3]}\t{lines[6]}")

Overwriting mapper_ex2.py


In [7]:
%%file reducer_ex2.py
#!/usr/bin/env python

import sys

# variable that will store the current iteration key
last_type = ""
# variable that will hold the regions for every key
regions = []

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    type_, region = line.split('\t')

    # check if it's the first iteration ever and update key variable
    if last_type == "":
        last_type = type_

    # if it's the same key as the previous one
    if type_ == last_type:
        # check if the region it's already registered
        if region not in regions:
            regions.append(region)

    # else print last type and refresh a new key iteration
    else:
        # print type and region of previous iteration
        print(f"{last_type}\t{regions}\n")
        # refresh variables for new iteration
        last_type = type_
        regions = [region]

# print type and region of last iteration
print(f"{last_type}\t{regions}\n")

Overwriting reducer_ex2.py


In [8]:
!rm -rf results_types; bash -c 'time hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper_ex2.py,reducer_ex2.py -mapper mapper_ex2.py -reducer reducer_ex2.py -input natural_disasters_BIG.csv -output results_types'

2019-11-15 12:38:40,852 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2019-11-15 12:38:41,200 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2019-11-15 12:38:41,200 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2019-11-15 12:38:41,340 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2019-11-15 12:38:42,285 INFO mapred.FileInputFormat: Total input files to process : 1
2019-11-15 12:38:42,331 INFO mapreduce.JobSubmitter: number of splits:7
2019-11-15 12:38:43,020 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local130498968_0001
2019-11-15 12:38:43,020 INFO mapreduce.JobSubmitter: Executing with tokens: []
2019-11-15 12:38:44,033 INFO mapred.LocalDistributedCacheManager: Localized file:/home/jovyan/work/mapper_ex2.py as file:/tmp/hadoop-jovyan/mapred/local/job_local130498968_0001_160fd4be-7cbf-431a-8b33-c76bee7810f4/mapper_ex2.py
2019-11-15 12:38:44,132 INFO mapred.LocalD

In [9]:
!cat results_types/part-*

Animal accident	['Western Africa']
	
Complex Disasters	['Southern Asia', 'Western Asia', 'Western Africa', 'Eastern Africa', 'Eastern Asia', 'Central America', 'Russian Federation', 'Northern Africa', 'Middle Africa', 'South-Eastern Asia']
	
Drought	['Northern Africa', 'South-Eastern Asia', 'Western Africa', 'Eastern Africa', 'Eastern Asia', 'Western Asia', 'Southern Asia', 'Middle Africa', 'Central America', 'South America', 'Southern Europe', 'Caribbean', 'Eastern Europe', 'Southern Africa', 'Northern America', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Russian Federation', 'Northern Europe', 'Central Asia', 'Western Europe']
	
Earthquake	['Western Asia', 'South-Eastern Asia', 'Eastern Asia', 'Melanesia', 'Western Europe', 'Southern Europe', 'Southern Asia', 'South America', 'Northern America', 'Western Africa', 'Central America', 'Russian Federation', 'Northern Africa', 'Australia and New Zealand', 'Northern Europe', 'Southern Africa', 'Eastern Africa', 'M

### 3. What are the probabilities of getting injured or dying in a natural disaster of type T in the continent C during decade D (190x, 191x, 192x, ..., 199x, 200x, 201x)?

In [10]:
%%file mapper_ex3.py
#!/usr/bin/env python

# import sys
import sys
# import string library function  
import string  

# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split into columns
    lines = line.split(",")
    
    # filter rows that don't contain the required information
    if len(lines[3])>0 and len(lines[5]) and len(lines[10])>0 and len(lines[11])>0 and len(lines[14])>0:
        # key consists of decade (year with last digit as 0), continent and disaster type
        key = f"{lines[0][:len(lines[0])-1]}0,{lines[3]},{lines[5]}"
        # value consists of number of deaths, injured and total affected
        value = f"{lines[10]},{lines[11]},{lines[14]}"
        # print key and value
        print(f"{key}\t{value}")
    

Overwriting mapper_ex3.py


In [11]:
%%file reducer_ex3.py
#!/usr/bin/env python

import sys

# variable that will store the current iteration key
last_key = ""
# variables that will hold the count for deaths, injured and affected through iterations
total_deaths = 0
total_injured = 0
total_affected = 0

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    key, value = line.split("\t",1)
    # parse value inputs and cast them as integer
    deaths, injured, affected = value.split(",")
    deaths = int(deaths)
    injured = int(injured)
    affected = int(affected)

    # check if it's the first iteration ever and update key variable
    if last_key == "":
        last_key = key
        
    # if it's the same key as previous, update count variables
    if key == last_key:
        total_deaths += deaths
        total_injured += injured
        total_affected += affected      
        
    # else print previous iteration and refresh variables
    else:
        # compute death probability
        death_prob = total_deaths / total_affected
        # compute injured probability
        injured_prob = total_injured / total_affected
        # print last key and probabilities
        print(f"{(last_key, [death_prob,injured_prob])}\n")
    
        # refresh variables for new iteration
        last_key = key
        total_deaths = deaths
        total_injured = injured
        total_affected = affected   
    
# compute death probability
death_prob = total_deaths / total_affected
# compute injured probability
injured_prob = total_injured / total_affected
# print key and probabilites for last iteration
print(f"{(last_key, [death_prob,injured_prob])}\n")

Overwriting reducer_ex3.py


In [12]:
!rm -rf results_affected; bash -c 'time hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper_ex3.py,reducer_ex3.py -mapper mapper_ex3.py -reducer reducer_ex3.py -input natural_disasters_BIG.csv -output results_affected'

2019-11-15 12:39:25,838 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2019-11-15 12:39:26,238 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2019-11-15 12:39:26,238 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2019-11-15 12:39:26,334 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2019-11-15 12:39:27,192 INFO mapred.FileInputFormat: Total input files to process : 1
2019-11-15 12:39:27,271 INFO mapreduce.JobSubmitter: number of splits:7
2019-11-15 12:39:28,136 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local186253206_0001
2019-11-15 12:39:28,137 INFO mapreduce.JobSubmitter: Executing with tokens: []
2019-11-15 12:39:30,191 INFO mapred.LocalDistributedCacheManager: Localized file:/home/jovyan/work/mapper_ex3.py as file:/tmp/hadoop-jovyan/mapred/local/job_local186253206_0001_355f4e08-c12e-4fd3-8743-74dd3b880390/mapper_ex3.py
2019-11-15 12:39:30,346 INFO mapred.LocalD

In [13]:
!cat results_affected/part-*

('1900,Mass movement (dry),Americas', [2.3806794924273436, 0.9582480556692591])	
	
('1900,Miscellaneous accident,Europe', [8.976744186046512, 0.872093023255814])	
	
('1910,Earthquake,Asia', [0.5596336192994904, 0.9267453611129028])	
	
('1910,Landslide,Americas', [2.6968130921619293, 1.0645994832041343])	
	
('1910,Storm,Americas', [0.011396990133858885, 0.08069502956226156])	
	
('1910,Transport accident,Americas', [0.11703810858460287, 0.6026518649591748])	
	
('1910,Transport accident,Europe', [0.9500691180536356, 0.9187724633674316])	
	
('1910,Volcanic activity,Asia', [7.011273957158963, 0.9954904171364148])	
	
('1920,Earthquake,Asia', [0.6777652879787639, 0.5234847966759577])	
	
('1920,Industrial accident,Europe', [0.39007808821221157, 1.049697849631141])	
	
('1920,Storm,Asia', [0.017491903516491103, 0.00026905528495035246])	
	
('1920,Transport accident,Europe', [0.2579578338156263, 0.8865233567589913])	
	
('1930,Earthquake,Americas', [0.5168817378069559, 1.0585023858268818])	
	
('193

### 4. Optional Exercise. What is the mean total damage and most costly disaster's expanses of a certain disaster subgroup in each country?

In [14]:
%%file mapper_ex4.py
#!/usr/bin/env python

# import sys
import sys
# import string library function  
import string  

# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split into columns
    lines = line.split(",")
    
    # filter rows that don't contain the required information
    if len(lines[15])>0:
        # key consists of country and disaster subgroup
        key = f"{lines[8]},{lines[2]}"
        # value consists of total damage and occurrences
        value = f"{lines[15]},{lines[9]}"
        # print key and value
        print(f"{key}\t{value}")
    

Writing mapper_ex4.py


In [15]:
%%file reducer_ex4.py
#!/usr/bin/env python

import sys

# variable that will store the current iteration key
last_key = ""
# variables that will hold the count for total damage, occurrences and maximum damage
total_damage = 0
total_occurrences = 0
maximum_damage = 0

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    key, value = line.split("\t",1)
    # parse value inputs and cast them as integer
    damage, occurrences = value.split(",")
    damage = float(damage)
    occurrences = int(occurrences)

    # check if it's the first iteration ever and update key variable
    if last_key == "":
        last_key = key
        
    # if it's the same key as previous, update count variables
    # update maximum damage if current damage is higher
    if key == last_key:
        total_damage += damage
        total_occurrences += occurrences
        if damage > maximum_damage:
            maximum_damage = damage
        
    # else print previous iteration and refresh variables
    else:
        # compute mean damage
        mean_damage = total_damage / total_occurrences
        # print last key and expanses
        print(f"{(last_key, [mean_damage,maximum_damage])}\n")
    
        # refresh variables for new iteration
        last_key = key
        total_damage = damage
        total_occurrences = occurrences
        maximum_damage = damage
    
# compute mean damage
mean_damage = total_damage / total_occurrences
# print last key and expanses for last iteration
print(f"{(last_key, [mean_damage,maximum_damage])}\n")

Writing reducer_ex4.py


In [16]:
!rm -rf results_costs; bash -c 'time hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper_ex4.py,reducer_ex4.py -mapper mapper_ex4.py -reducer reducer_ex4.py -input natural_disasters_BIG.csv -output results_costs'

2019-11-15 12:40:16,321 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2019-11-15 12:40:16,726 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2019-11-15 12:40:16,727 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2019-11-15 12:40:16,799 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2019-11-15 12:40:17,802 INFO mapred.FileInputFormat: Total input files to process : 1
2019-11-15 12:40:17,886 INFO mapreduce.JobSubmitter: number of splits:7
2019-11-15 12:40:18,478 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local73766485_0001
2019-11-15 12:40:18,486 INFO mapreduce.JobSubmitter: Executing with tokens: []
2019-11-15 12:40:19,518 INFO mapred.LocalDistributedCacheManager: Localized file:/home/jovyan/work/mapper_ex4.py as file:/tmp/hadoop-jovyan/mapred/local/job_local73766485_0001_1f2848ec-bf9f-4aaf-99b0-1b69e747adbf/mapper_ex4.py
2019-11-15 12:40:19,619 INFO mapred.LocalDis

In [17]:
!cat results_costs/part-*

('Afghanistan,Climatological', [48317.25543478261, 142000.0])	
	
('Afghanistan,Geophysical', [5556.998299319728, 25000.0])	
	
('Afghanistan,Hydrological', [28878.731343283584, 260000.0])	
	
('Afghanistan,Meteorological', [1605.6077348066299, 5000.0])	
	
('Albania,Hydrological', [8316.229213483146, 17500.0])	
	
('Algeria,Geophysical', [1139831.8787593986, 5200000.0])	
	
('Algeria,Hydrological', [98957.39966555184, 779000.0])	
	
('Algeria,Technological', [800000.0, 800000.0])	
	
('American Samoa,Hydrological', [50000.0, 50000.0])	
	
('American Samoa,Meteorological', [76846.84684684685, 150000.0])	
	
('Angola,Hydrological', [5562.753036437247, 10000.0])	
	
('Anguilla,Meteorological', [50843.74605678234, 200000.0])	
	
('Antigua and Barbuda,Meteorological', [130494.42176870749, 350000.0])	
	
('Argentina,Climatological', [1304880.9523809524, 3400000.0])	
	
('Argentina,Geophysical', [92530.97345132743, 104000.0])	
	
('Argentina,Hydrological', [396252.17713270144, 1800000.0])	
	
('Argentina,Me